In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import random


In [2]:
new_train = pd.read_csv("efficiency - Sheet1.csv")

In [3]:
new_train

,Inlet velocity,Offset distance,Number of airfoils,Net Pressure (Pa),Surface Heat Transfer Coef. On contact_region_2-trg ([W/(m^2 K)]),Efficiency
0,1.757,30000,1,7312.80850,7214.59110,1.000000
1,1.757,30000,5,9053.63475,8235.67825,1.184612
2,1.757,30000,10,10794.46100,9256.76540,1.358421
3,1.757,30000,15,11818.25600,9964.18050,1.451195
4,1.757,30000,20,13214.09800,11124.17300,1.564112
...,...,...,...,...,...,...
98,2.510,40000,25,29090.95000,15807.53000,1.270131
99,2.510,40000,30,30781.64000,15606.01000,1.349707
100,2.510,40000,35,30781.64000,15606.01000,1.349707
101,2.510,40000,40,30898.06000,15046.73000,1.371394


In [4]:
new_train = new_train.sample(frac=1)

In [5]:
new_train.drop(["Net Pressure (Pa)" ,"Surface Heat Transfer Coef. On contact_region_2-trg ([W/(m^2 K)])"] , axis =1 , inplace = True)

In [6]:
new_train

,Inlet velocity,Offset distance,Number of airfoils,Efficiency
86,2.008,35000,20,1.182109
3,1.757,30000,15,1.451195
76,2.510,35000,20,1.230925
21,1.757,40000,20,1.302195
31,1.757,45000,20,1.380699
...,...,...,...,...
96,2.510,40000,15,1.134025
37,1.506,30000,15,1.209910
85,2.008,35000,15,1.232932
93,2.510,40000,1,1.000000


In [152]:
x = new_train[["Inlet velocity" , "Offset distance" , "Number of airfoils"]].values

In [153]:
y = new_train["Efficiency"].values

In [193]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature matrix 'X' and target vector 'y'

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

# Create an SVR model with a non-linear kernel (e.g., Radial Basis Function - RBF)
svr_model = SVR(kernel='rbf')

# Fit the SVR model to the scaled data
svr_model.fit(X_scaled, y)

# Predict on new data
X_test_scaled = scaler.transform(x)
y_pred = svr_model.predict(X_test_scaled)



In [194]:
y_pred

array([1.27085242, 1.36479678, 1.16911264, 1.27650574, 1.34430292,
       1.38797915, 1.44218219, 1.27235188, 1.3594506 , 1.42854738,
       1.12845705, 1.36436231, 1.09988316, 1.18725943, 1.13587486,
       1.22662732, 1.08001191, 1.0374795 , 1.08974308, 1.19454469,
       1.30263928, 1.25768809, 1.47698988, 1.02533294, 1.1860714 ,
       1.4615955 , 1.06783483, 1.20256207, 1.14654448, 1.0883137 ,
       1.30795644, 1.39976737, 1.46399723, 1.37192862, 1.36935877,
       1.09684255, 1.47597029, 1.099636  , 1.27731064, 1.10608144,
       1.23620074, 1.04575769, 1.0389697 , 1.05053422, 1.44981277,
       1.10015713, 1.45865532, 1.43211825, 1.29060525, 1.26356833,
       1.50184144, 1.39330774, 1.50859903, 1.07931355, 1.11779247,
       1.30598972, 1.30998277, 1.14881684, 1.49112269, 1.3663602 ,
       1.29897009, 1.30910319, 1.2736095 , 1.36342687, 1.09958123,
       1.13093907, 1.30783467, 1.00911875, 1.51522714, 1.22871367,
       1.1604774 , 1.05986194, 1.52891882, 1.4971824 , 1.18037

In [236]:
num_weights = 3
pop_size = 30
v_range = (1.6, 1.8)
d_range = (30000, 45000)
n_range = (15 , 30)

# Initialize population
new_population = []
for i in range(pop_size):
    vel = np.random.uniform(*v_range)
    offset = np.random.uniform(*d_range)
    airfoils = np.random.randint(*n_range)
    new_population.append([vel, offset, airfoils])

new_population = np.array(new_population)

In [237]:
def fitness(new_population):
    X_test_scaled = scaler.transform(new_population)
    fitness = svr_model.predict(X_test_scaled)
    return fitness

In [238]:
def selection(new_population, fitness_values, num_parents):
    parents = np.empty((num_parents, new_population.shape[1]))

    for parent_num in range(num_parents):
        max_fitness_idx = np.argmax(fitness_values)
        parents[parent_num, :] = new_population[max_fitness_idx, :]
        fitness_values[max_fitness_idx] = -99999999999

    return parents

In [239]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.uint8(offspring_size[1] / 2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k % parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k + 1) % parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

In [240]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 2] = offspring_crossover[idx, 2] + random_value

    return offspring_crossover

In [241]:
num_generations = 50
num_parents_mating = 4

for generation in range(num_generations):
    fitness_values = calculate_fitness(new_population)
    parents = selection(new_population, fitness_values, num_parents_mating)
    offspring_crossover = crossover(parents, offspring_size=(pop_size - parents.shape[0], num_weights))
    offspring_mutation = mutation(offspring_crossover)
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    # The best result in the current iteration.
    print(f"efficiency : {fitness(new_population)} for case: {new_population}")

efficiency : [1.61944684 1.52639205 1.58727871 1.52199824 1.53448669 1.57018877
 1.51033905 1.61893596 1.52576999 1.58731722 1.52282376 1.62108928
 1.54843124 1.56949471 1.50712371 1.62456909 1.5285036  1.58885126
 1.52957566 1.61077305 1.53627008 1.57225429 1.52929132 1.61050851
 1.54766108 1.57641417 1.51744109 1.60937996 1.52723716 1.57442025] for case: [[1.78396154e+00 3.01178577e+04 2.90000000e+01]
 [1.67690929e+00 3.06157870e+04 2.40000000e+01]
 [1.70629825e+00 3.02795121e+04 2.70000000e+01]
 [1.79988472e+00 3.10915672e+04 2.40000000e+01]
 [1.78396154e+00 3.06157870e+04 2.38825958e+01]
 [1.67690929e+00 3.02795121e+04 2.63130100e+01]
 [1.70629825e+00 3.10915672e+04 2.34793411e+01]
 [1.79988472e+00 3.01178577e+04 2.91108133e+01]
 [1.78396154e+00 3.06157870e+04 2.34103736e+01]
 [1.67690929e+00 3.02795121e+04 2.76530446e+01]
 [1.70629825e+00 3.10915672e+04 2.41816771e+01]
 [1.79988472e+00 3.01178577e+04 2.93170574e+01]
 [1.78396154e+00 3.06157870e+04 2.46669041e+01]
 [1.67690929e+00 

In [242]:
fitness = fitness(new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])

Best solution :  [[[1.78396154e+00 3.01178577e+04 3.00158400e+01]]]
Best solution fitness :  [1.62935201]
